# Pipeline 2

Produce top 5 entites + gold entities and 0 relations from falcon

In [1]:
from pipeline import Falcon2Linker, SerialAnnotator, T5Converter
import time
from pprint import pprint

[Flacon2.0 main.py]: no reranking, k=5


In [2]:
from pathlib import Path
import pandas as pd
import json

data_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "train.json"
data2_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "test_KG_4211.json"
output_links_dir = Path("..") / "t5-for-sparql" / "falcon_links" / "5ents-gold_0rels"

In [3]:
linker = Falcon2Linker()
annotator = SerialAnnotator()
converter = T5Converter()

In [4]:
utterance = "Who is the wife of Barack Obama"

In [5]:
linked = linker.link(utterance)
# rules = [1,2,3,4,5,8,9,10,12,13,14]
# linked = process_text_E_R(utterance, rules)
pprint("Linked")
pprint(linked)

pprint("Annotated")
annotated = annotator.annotate(**linked)
pprint(annotated)

pprint("Converted")
converted = converter.preprocess_inputs(**annotated)
pprint(converted)

recieved utterance
0 : ['Who is the wife of Barack Obama']


['Who is the wife of Barack Obama', [['<http://www.wikidata.org/entity/P26>', 'wife'], ['<http://www.wikidata.org/entity/P2848>', 'wife'], ['<http://www.wikidata.org/entity/P140>', 'wife'], ['<http://www.wikidata.org/entity/P451>', 'wife'], ['<http://www.wikidata.org/entity/P600>', 'wife']], [['<http://www.wikidata.org/entity/Q76>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q649593>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q4808526>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q4858106>', 'Barack obama'], ['<http://www.wikidata.org/entity/Q643049>', 'Barack obama']], 0, 0, 0, 0]
'Linked'
{'ents': [{'id': 'Q76',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org/entity/Q76'},
          {'id': 'Q649593',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org/entity/Q649593'},
          {'id': 'Q4808526',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org/entity/Q4808526'},
          {'id': 'Q4858106',
  

{'fragments': ['[DEF]',
               'wd:',
               'Q76 Barack',
               '[DEF]',
               'wd:',
               'Q649593 Barack Hussein Obama, Sr.',
               '[DEF]',
               'wd:',
               'Q4808526 Obama assassination plot',
               '[DEF]',
               'wd:',
               'Q4858106 Barack Obama Academy',
               '[DEF]',
               'wd:',
               'Q643049 family of Barack Obama',
               '[DEF]',
               'wdt:',
               'P26 partner',
               '[DEF]',
               'wdt:',
               'P2848 wifi',
               '[DEF]',
               'wdt:',
               'P140 religion',
               '[DEF]',
               'wdt:',
               'P451 common-law spouse',
               '[DEF]',
               'wdt:',
               'P600 Wine AppDB-ID'],
 'utterance': 'Who is the wife of Barack Obama'}
'Converted'
('Who is the wife of Barack Obama <extra_id_59> <extra_id_53> Q76 Barack '

In [6]:
import re

def retrieve_gold_links(masked_wikisparql):
  fragments = masked_wikisparql.upper().strip().split(' ')
  ents = []
  rels = []
  result_dict = {
    'ents': ents,
    'rels': rels,
  }
  for fragment in fragments:
    id_match = re.search('[QP][0-9]+',fragment)
    if id_match:
      id_raw = id_match.group(0)
      if id_raw[0] == 'Q':
        prefix = 'wdt:'
        uri = 'http://www.wikidata.org/entity/' + id_raw
        target_arr = ents
      elif id_raw[0] == 'P':
        prefix = 'wd:'
        uri =  'http://www.wikidata.org/prop/direct/' + id_raw
        target_arr = rels
      else:
        raise ValueError('ID does not start with P or Q')
      match_dict = {
        'id' : id_raw,
        'prefix' : prefix,
        'uri' : uri,
      }
      target_arr.append(match_dict)
  return result_dict

In [7]:
def pipe(utterance, wikisparql):
    linked = linker.link(utterance)
    annotated = annotator.annotate(**linked)
    converted = converter.preprocess(**annotated, wikisparql=wikisparql)
    return linked, annotated, converted

In [8]:
def pipe_batch(utterances, wikisparqls):
  batched = []
  link_batch_start = time.time()
  linked = linker.link_batch(utterances)

  assert len(linked) == len(wikisparqls)

  for i, link in enumerate(linked):
    gold_linked = retrieve_gold_links(wikisparqls[i])
    new_ents=link['ents']
    old_ents_set = set(x['id'] for x in link['ents'])
    for gold_ent in gold_linked['ents']:
        if gold_ent['id'] not in old_ents_set:
            new_ents.append(gold_ent)
    link['rels'] = []
    assert len(linked[i]['rels']) == 0

  link_batch_end = time.time()
  s = time.time()
  annotated = annotator.batch_annotate(linked)
  e = time.time()
  assert len(linked) == len(annotated)
  annotator_time = e - s
  converter_time = 0
  for i, single_annotated in enumerate(annotated):
    # annotated = annotator.annotate(**single_linked)
    # try:
    #   annotator_time += e - s
    # except Exception as e:
    #   print("[Annotator Error]:", e)
    #   continue
    s = time.time()
    try:
      converted = converter.preprocess(**single_annotated, wikisparql=wikisparqls[i])
    except Exception as e:
      print("[Converter Error]:", e)
      continue
      
    e = time.time()
    converter_time += e - s
    batched.append([linked[i], annotated[i], converted])
  print("Link batch time:", link_batch_end - link_batch_start)
  print("Anno batch time:", annotator_time)
  print("Conv batch time:", converter_time)
  return batched

## Data

In [9]:
with open(data_path) as f:
  data_dict = json.load(f)

with open(data2_path) as f:
  data_dict.extend(json.load(f))

count = 0

for item in data_dict:
  paraphrased_question = item["paraphrased_question"]
  question = item["question"]
  if paraphrased_question == None:
    raise Exception
  if question == None:
    count += 1
    # print(len(paraphrased_question) > 2)
    # pprint(item)
  if item["sparql_wikidata"] == None:
    raise Exception
  

  if len(paraphrased_question) > 2000:
    paraphrased_question = question
  # item["new_question"] = paraphrased_question if len(paraphrased_question) > 2 and paraphrased_question != [] else question
  item["new_question"] = question

print("Missing question count:", count)

df = pd.DataFrame.from_dict(data_dict)
df = df.dropna(axis=0, subset=['new_question'])
df = df[~df.question.isin(['n/a', None, 'none', 'None'])]
n_entries = len(df)
print("Number of rows:", n_entries)

Missing question count: 71
Number of rows: 28247


In [10]:
assert(len(df[df.question.isna()]) == 0)

In [11]:
for data in df[0:5].iterrows():
    print(data[1].new_question)

What periodical literature does Delta Air Lines use as a moutpiece?
Who is the child of Ranavalona I's husband?
Is it true Jeff_Bridges occupation Lane Chandler and photographer ?
What is the pre-requisite of phase matter of Galinstan?
Which is the operating income for Qantas?


In [12]:
responses = []
batch_qns = []
batch_ans = []
truncated_data = df[:]
total_len = len(truncated_data)
last = 0
cutoff = 0
for i, data in enumerate(truncated_data.iterrows()):
    data = data[1]
    if i < cutoff:
        continue
    # print("Pipeline iter", i)
    question = data["new_question"]
    answer = data["sparql_wikidata"]
    # try
    #     linked, annotated, converted = pipe(question, answer)
    # except Exception as e:
    #     print(e)
    #     continue
    # responses.append([linked, annotated, converted])
    if (question and answer) and len(question) >= 2:
        batch_qns.append(question)
        batch_ans.append(answer)
    if ((i + 1) % 50) == 0 or i == total_len - 1:
        print("\n[Pipeline2]:", f"Linking {last}-{i}")
        try:
            for linked, annotated, converted in pipe_batch(batch_qns, batch_ans):
                responses.append([linked, annotated, converted])
            batch_qns = []
            batch_ans = []
            last = i
        except Exception as e:
            raise e
            # print(e)
            # continue
        with open(output_links_dir / f"link_{i}.json", "w") as f:
            json.dump(responses, f, indent=2, separators=(',',':'))
        print()


[Pipeline2]: Linking 0-49
1 : ['What periodical literature does Delta Air Lines use as a moutpiece?']
1 : ['Who is the child of Ranavalona Is husband?']
1 : ['Is it true Jeff_Bridges occupation Lane Chandler and photographer ?']
1 : ['What is the pre-requisite of phase matter of Galinstan?']
1 : ['Which is the operating income for Qantas?']
1 : ['which cola starts with the letter p']
1 : ['Is the right ascension of malin 1 less than 15.1398?']
1 : ['What is the complete list of records released by Jerry Lee Lewis?']
1 : ['What is Mary Lou Rettons International Olympic Committee athlete ID.']
1 : ['Who won the prize at the sequel of the 1885 Wimbledon Championships- Gentlemens Singles?']
1 : ['Is it true that the carbon footprint of the iPhone X Max is 106?']
1 : ['Tell me the female beauty pageant that operates in all countries and contains the word model in its name?']
1 : ['Did Lindsey Vonn participate in the FIS Alpine World Ski Championships in 1999 and 2019?']
1 : ['When position

2 : ['Who is the student that coined the Euler-Lagrange equation?']


3 : ['Which website does Twitch own?']


4 : ['Is the minimal lethal dose of the benzene equal to 170000?']
5 : ['Who is the person that was a student of Ivan Pavlov?']
6 : ['is the clock speed of the Watara Supervision smaller than 3.2?']


7 : ['What country is the current leader of the African Union ?']


...

864 : ['Is The New York Times published in Manhattan, New York City?']
865 : ['Who is the child of William Jennings Bryan that has the biological sex of male?']


866 : ['What U.S. Government agency holds the archives of Jefferson Davis?']


867 : ['What book is the most notable work of Robert Louis Stevenson?']


868 : ['What kind of instrument is made with foil?']
869 : ['What is the reservoir of Lake Winnebago?']


Link batch time: 12.596404075622559
Anno batch time: 264.59366631507874
Conv batch time: 0.0038580894470214844


[Pipeline2]: Linking 899-949
901 : ['Which fruit type is the higher taxon of Philodendron edmundoi?']
901 : ['How many countries are around Ireland?']
901 : ['Which song was Ringo Starr nominated for the Academy Award for Best Original Song Score?']
901 : ['What is the language official in Federated States of Micronesia having a shortened name of']
901 : ['This sentence makes no sense.']
901 : ['When Camille Pissarro ended to work at Netherlands?']
901 : ['What killed John F. Kennedy and who caused him ballistic trauma?']
901 : ['For which work Patrick White received the Miles Franklin Literary Award?']
901 : ['When Alexandria, Diocese of Egypt, ended?']
901 : ['Who owns Disneyland and who is it named after?']
901 : ['What are the treaty which start with the letter t']
901 : ['Is it true that the caliber of Ehrhardt 7.5 cm Model 1901 equals to 60?']
901 : ['What is the item 